# Criação de dados

A proposta deste notebook é criar um dataframe de dados mockados contendo as seguintes colunas:
- id
- data
- usuário
- empresa

In [60]:
import pandas as pd
from faker import Faker
import uuid
from datetime import datetime
from random import choice

In [13]:
fake = Faker(["pt_BR"])

In [29]:
# Quantidade de registros na tabela

lines = 100_000

In [74]:


# Criação de dados 

base_company_list = [fake.unique.company_id() for _ in range(int(lines/100))] # 1k 
base_user_list = [fake.unique.email() for _ in range(int(lines/5))] # 20k
user_company = [(base_user, choice(base_company_list)) for base_user in base_user_list]
full_user_company = [choice(user_company) for _ in range(lines)]
# deste modo eu garanto que sempre um mesmo user vai pertencer a uma mesma empresa


data = {
    "id": [uuid.uuid4() for _ in range(lines)],
    "insert_date": [fake.date_time_between(start_date=datetime(2023, 1, 1)) for _ in range(lines)],
    "username": [user[0] for user in full_user_company],
    "company": [company[1] for company in full_user_company],
}


In [75]:
# Criação do dataframe

df = pd.DataFrame(data)

In [76]:
df.head()

,id,insert_date,username,company
0,daa4761b-c438-4841-8622-0682816c4674,2023-02-16 05:37:48,eduardoda-mata@example.org,75930264000143
1,3c374f33-9c12-4293-96ff-bde89d5834da,2023-08-24 16:31:38,barbosaantonio@example.com,16230487000139
2,5eae9ca5-0f02-462a-ad8d-450c0fe425ef,2023-01-15 19:26:53,pedro-lucas03@example.org,30514827000184
3,90ebe682-0e93-42f7-863b-4300198024fc,2023-02-14 05:17:19,stella89@example.org,87153642000109
4,0c6672de-c3bf-44c4-941d-c2eccf173581,2023-03-10 01:50:47,camposkamilly@example.com,62853704000164


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   id           100000 non-null  object        
 1   insert_date  100000 non-null  datetime64[ns]
 2   username     100000 non-null  object        
 3   company      100000 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 3.1+ MB


In [78]:
# Gerando o arquivo SQL

# Este foi o filename que usei ao rodar no meu pc para criar o script
filename = "../docker_databases/init.sql"

# Estou deixando este apenas para que possa executar no seu computador sem comprometer o resultado
# das analises que realizei.
# filename = "../docker_databases/init_avaliacao.sql"

with open(filename, "w") as f:
    f.write("\n\n-- Criação da tabela")
    f.write("""
    CREATE TABLE t_ponto (
          id VARCHAR(255) PRIMARY KEY,
          insert_date TIMESTAMP,
          username VARCHAR(255),
          company VARCHAR(255)
    );\n
    """)
    
    f.write("\n\n-- Exportação dos dados\n\n")
    for i in df.index:
        f.write(f"INSERT INTO t_ponto (id, insert_date, username, company) VALUES ('{str(df.iloc[i, 0])}', '{df.iloc[i, 1]}', '{df.iloc[i, 2]}', '{df.iloc[i, 3]}');\n")

In [67]:
print("Fim")

Fim
